# 패키지들은 연습 문제를 풀며 연습하기 위해 별도 로드 X

# Question 1.
- 서비스 이탈예측 데이터.
- 2 유형 환경과 일치시키기 위해 y_train과 x_train을 하나의 train 데이터 셋으로 병합하여 진행.

## 데이터 병합 및 확인 단계

In [43]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')

train = pd.concat([x_train, y_train['Exited']], axis = 1)
display(train.info())
display(x_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
 12  Exited           6499 non-null   int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 660.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3501 entries, 0 to 3500
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       3501 non-null   int64  
 1   Surname          3501 non-null   object 
 2   CreditScore      3501 non-null   int64  
 3   Geography        3501 non-null   object 
 4   Gender           3501 non-null   object 
 5   Age              3501 non-null   int64  
 6   Tenure           3501 non-null   int64  
 7   Balance          3501 non-null   float64
 8   NumOfProducts    3501 non-null   int64  
 9   HasCrCard        3501 non-null   int64  
 10  IsActiveMember   3501 non-null   int64  
 11  EstimatedSalary  3501 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 328.3+ KB


None

In [44]:
# 1. 데이터의 통계량 확인
display(train.describe(include = ['object', 'category']))
display(train.describe())

# 결과 변수
#   - count: 결측치를 제외한 전체 데이터의 수 (행의 수와 일치).
#   - unique: 서로 다른 범주의 수 (범주 종류의 수).
#   - top: 최빈 범주
#   - freq: 최빈 범주의 개수.

,Surname,Geography,Gender
count,6499,6499,6499
unique,2289,3,4
top,Brown,France,Male
freq,21,3227,3485


,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,6.499000e+03,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000
mean,1.569157e+07,650.396830,38.957070,5.041545,76836.581068,1.519772,0.708878,0.514387,100346.564524,0.203724
std,7.187584e+04,96.618957,10.502803,2.891779,62407.570894,0.578975,0.454314,0.499831,57944.655305,0.402797
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,1.562949e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,50907.565000,0.000000
50%,1.569181e+07,651.000000,37.000000,5.000000,97560.160000,1.000000,1.000000,1.000000,100496.840000,0.000000
75%,1.575358e+07,718.000000,44.000000,8.000000,127844.690000,2.000000,1.000000,1.000000,150480.155000,0.000000
max,1.581566e+07,850.000000,92.000000,10.000000,238387.560000,4.000000,1.000000,1.000000,199970.740000,1.000000


## 변수 선택 단계

In [45]:
# Surname과 CustomerId는 제외
#   - Surname: 범주의 종류가 2289개로 의미없는 정보임.
#   - CustomerId: Id 컬럼이기에 제외
train_df = train.drop(columns = ['Surname', 'CustomerId'])
x_test_df = x_test.drop(columns = ['Surname', 'CustomerId'])

## 설명변수와 결과변수 분리

In [46]:
# train_df의 설명변수와 결과변수를 분리
X = train_df.drop(columns = 'Exited')
y = train_df['Exited']

## Encoding

In [52]:
# 범주형 변수만 따로 추출
target = train_df.drop(columns = 'Exited')
cat_lst = target.select_dtypes(exclude = 'number').columns.tolist()
cat_lst

# 원-핫 인코딩 적용
#   - drop_first = False
#       - False: 기준 범주 없이 모든 범주를 0과 1로 encoding 할 때 사용한다. 이는 기준 범주 없이 개별 범주로 처리하기 때문.
#       - True: 통계모델에서 기준 범주를 설정할 때 사용한다. 통계에서는 기준 범주를 기준으로하여 변화량을 측정하기 때문. (이 경우 기준범주가 0이 된다).
train_encoded = pd.get_dummies(X, columns = cat_lst, drop_first = False)
x_test_encoded = pd.get_dummies(x_test_df, columns = cat_lst, drop_first = False)

## 데이터를 분리하고 학습결과 확인.

In [53]:
from sklearn.model_selection import train_test_split

# train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_encoded, y, test_size = 0.3, random_state = 42, stratify = y)

## 학습

In [54]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 결과 지표들 확인

In [55]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1. 예측
val_pred_label = rf.predict(X_val)               # 클래스 예측
val_pred_proba = rf.predict_proba(X_val)[:, 1]  # 양성 클래스 확률 (AUC에 사용)

# 2. 각 지표 출력
print("📊 모델 성능 평가 결과:")
print(f"Accuracy : {accuracy_score(y_val, val_pred_label):.4f}")
print(f"Precision: {precision_score(y_val, val_pred_label):.4f}")
print(f"Recall   : {recall_score(y_val, val_pred_label):.4f}")
print(f"F1-score : {f1_score(y_val, val_pred_label):.4f}")
print(f"AUC-ROC  : {roc_auc_score(y_val, val_pred_proba):.4f}")

# 3. 추가 보고서
print("\n📋 Classification Report:")
print(classification_report(y_val, val_pred_label))

# 4. 혼동행렬
print("🧮 Confusion Matrix:")
print(confusion_matrix(y_val, val_pred_label))


📊 모델 성능 평가 결과:
Accuracy : 0.8605
Precision: 0.7962
Recall   : 0.4232
F1-score : 0.5526
AUC-ROC  : 0.8467

📋 Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1553
           1       0.80      0.42      0.55       397

    accuracy                           0.86      1950
   macro avg       0.83      0.70      0.74      1950
weighted avg       0.85      0.86      0.84      1950

🧮 Confusion Matrix:
[[1510   43]
 [ 229  168]]


## test 데이터에 적용하여 csv파일 생성.

In [57]:
test_pred_label = rf.predict(x_test_encoded)
test_pred_proba = rf.predict_proba(x_test_encoded)[ : , 1]

# ID 생성
pd.DataFrame({'pred' : test_pred_label}).to_csv('label.csv', index=True, index_label = 'ID')
pd.DataFrame({'pred' : test_pred_proba}).to_csv('proba.csv', index=True, index_label = 'ID')

# Question 2.
- 이직여부 판단 데이터
- 병합된 데이터가 들어옴을 가정.

## 데이터 병합 및 확인

In [67]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv')

train_total = pd.concat([x_train, y_train['target']], axis = 1)
display(train_total.head())
display(x_test.head())

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45,0.0
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17,1.0
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50,0.0
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135,0.0
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17,1.0


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,7129,city_23,0.899,Male,No relevent experience,no_enrollment,Graduate,STEM,2,NaN,NaN,1,23
1,31037,city_44,0.725,Male,No relevent experience,Part time course,Masters,STEM,5,NaN,NaN,never,39
2,22179,city_103,0.920,Male,No relevent experience,Full time course,Graduate,STEM,7,NaN,NaN,2,262
3,29724,city_50,0.896,Male,No relevent experience,Full time course,Graduate,STEM,7,NaN,NaN,never,78
4,17977,city_149,0.689,Female,Has relevent experience,Full time course,Masters,STEM,19,50-99,Pvt Ltd,1,125


# 데이터의 정보확인

In [68]:
display(train_total.info())
display(x_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12452 non-null  int64  
 1   city                    12452 non-null  object 
 2   city_development_index  12452 non-null  float64
 3   gender                  9535 non-null   object 
 4   relevent_experience     12452 non-null  object 
 5   enrolled_university     12195 non-null  object 
 6   education_level         12137 non-null  object 
 7   major_discipline        10586 non-null  object 
 8   experience              12415 non-null  object 
 9   company_size            8600 non-null   object 
 10  company_type            8471 non-null   object 
 11  last_new_job            12179 non-null  object 
 12  training_hours          12452 non-null  int64  
 13  target                  12452 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6706 entries, 0 to 6705
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             6706 non-null   int64  
 1   city                    6706 non-null   object 
 2   city_development_index  6706 non-null   float64
 3   gender                  5115 non-null   object 
 4   relevent_experience     6706 non-null   object 
 5   enrolled_university     6577 non-null   object 
 6   education_level         6561 non-null   object 
 7   major_discipline        5759 non-null   object 
 8   experience              6678 non-null   object 
 9   company_size            4620 non-null   object 
 10  company_type            4547 non-null   object 
 11  last_new_job            6556 non-null   object 
 12  training_hours          6706 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 681.2+ KB


None

## 통계량 확인

In [69]:
display(train_total.describe())
display(train_total.describe(include = ['object', 'category']))

,enrollee_id,city_development_index,training_hours,target
count,12452.000000,12452.000000,12452.000000,12452.000000
mean,16966.301076,0.829709,65.604481,0.249358
std,9646.223269,0.123087,60.430340,0.432659
min,1.000000,0.448000,1.000000,0.000000
25%,8591.500000,0.740000,23.000000,0.000000
50%,17177.500000,0.910000,47.000000,0.000000
75%,25327.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


,city,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
count,12452,9535,12452,12195,12137,10586,12415,8600,8471,12179
unique,122,3,2,3,5,6,22,8,6,6
top,city_103,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1
freq,2839,8595,8953,8975,7540,9414,2179,2000,6413,5157


## 결측치 처리.
- 현재 object형 컬러만 결측치가 존재.

In [86]:
# object이거나 category형인 컬럼만 필터링.
cat_lst = train_total.select_dtypes(include = ['object', 'category']).columns
num_lst = train_total.select_dtypes(include = 'number').columns
train_cat = train_total.loc[ : , cat_lst]
train_num = train_total.loc[ : , num_lst]

# fillna(value, inplace = )를 통해 최빈값으로 대체
#   - x.mode(dropna = True)[0]: 최빈값 중 첫 번째 값을 반환해서 결측치 대신 채움
#       - dropna: True는 결측치를 제외한 최빈값을 반환.
#   - inplace: True인 경우 원본 데이터프레임을 변환함. False인 경우 새로운 데이터프레임을 반환함.
train_cat_nn = train_total[cat_lst].apply(lambda x: x.fillna(x.mode(dropna = True)[0], inplace = False))

# 결측값 처리 후 다시 수치형 변수들과 병합.
train_nn = pd.concat([train_cat_nn, train_num], axis = 1)

# 결과 확인
display(train_nn.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    12452 non-null  object 
 1   gender                  12452 non-null  object 
 2   relevent_experience     12452 non-null  object 
 3   enrolled_university     12452 non-null  object 
 4   education_level         12452 non-null  object 
 5   major_discipline        12452 non-null  object 
 6   experience              12452 non-null  object 
 7   company_size            12452 non-null  object 
 8   company_type            12452 non-null  object 
 9   last_new_job            12452 non-null  object 
 10  enrollee_id             12452 non-null  int64  
 11  city_development_index  12452 non-null  float64
 12  training_hours          12452 non-null  int64  
 13  target                  12452 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

None

## Encoding 적용

In [101]:
# 결과변수 분리
X = train_nn.drop(columns = 'target')
y = train_nn['target']

# get_dummies를 통한 인코딩
train_encoded = pd.get_dummies(X, columns = cat_lst, drop_first = False)
x_test_encoded = pd.get_dummies(x_test, columns = cat_lst, drop_first = False)

# Encoding 된 범주의 수가 일치하지 않는 경우 맞춰준다.
train_encoded, x_test_encoded = train_encoded.align(x_test_encoded, join='left', axis=1, fill_value=0)

## 데이터 분리

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_encoded, y, test_size = 0.3, random_state = 42)

## 학습

In [97]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 결과확인

In [98]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1. 예측
val_pred_label = rf.predict(X_val)               # 클래스 예측
val_pred_proba = rf.predict_proba(X_val)[:, 1]  # 양성 클래스 확률 (AUC에 사용)

# 2. 각 지표 출력
print("📊 모델 성능 평가 결과:")
print(f"Accuracy : {accuracy_score(y_val, val_pred_label):.4f}")
print(f"Precision: {precision_score(y_val, val_pred_label):.4f}")
print(f"Recall   : {recall_score(y_val, val_pred_label):.4f}")
print(f"F1-score : {f1_score(y_val, val_pred_label):.4f}")
print(f"AUC-ROC  : {roc_auc_score(y_val, val_pred_proba):.4f}")

# 3. 추가 보고서
print("\n📋 Classification Report:")
print(classification_report(y_val, val_pred_label))

# 4. 혼동행렬
print("🧮 Confusion Matrix:")
print(confusion_matrix(y_val, val_pred_label))

📊 모델 성능 평가 결과:
Accuracy : 0.7642
Precision: 0.5419
Recall   : 0.3733
F1-score : 0.4421
AUC-ROC  : 0.7530

📋 Classification Report:
              precision    recall  f1-score   support

         0.0       0.81      0.89      0.85      2801
         1.0       0.54      0.37      0.44       935

    accuracy                           0.76      3736
   macro avg       0.68      0.63      0.65      3736
weighted avg       0.74      0.76      0.75      3736

🧮 Confusion Matrix:
[[2506  295]
 [ 586  349]]


## CSV 파일 생성

In [100]:
test_pred_label = rf.predict(x_test_encoded)
test_pred_proba = rf.predict_proba(x_test_encoded)[ : , 1]

pd.DataFrame({'pred' : test_pred_label}).to_csv('label_csv', index = True, index_label = 'ID')
pd.DataFrame({'pred_proba' : test_pred_proba}).to_csv('proba_csv', index = True, index_label = 'ID')